# NEM Open Data Extraction Tool

**The Objective**: to create a tool that can easily extract open data from nemweb.com.au and assemble it into an analysis-ready format.

* For the intitial testing phase, extracted data will be stored as CSV files
* Once the data extraction pipeline is set up, we can experiment with SQL databases

In [1]:
%load_ext autoreload
%autoreload 2

# Standard Python
import os
from io import BytesIO
from zipfile import ZipFile
import pandas as pd

# My modules
from nem_tracker import NEM_tracker
import config
config.use('config.json')

Value for DATA_PATH has been set!


### Creating a custom JSON sitemap

We require a dynamic way of navigating the NEM website to locate and extract resources. A JSON file that tracks the data files in a given location on the NEM website could be helpful... 

* Define data directories i.e. a function to add new directories to JSON
  * Perhaps have a *comments* field that allows user to insert comments about the data resource
* Each data directory file has an additional flag whether the data has been extracted or not - maybe a CSV file might work better... (it could also convert the timestamps to datetime to allow datatime queries)

In [2]:
data_dir = os.getenv('DATA_PATH')
nem_trk = NEM_tracker(data_dir)
resource = '/Reports/Current/Operational_Demand/ACTUAL_DAILY/'
tracker_dir = nem_trk.tracker_dir
test = pd.read_csv(os.path.join(tracker_dir, os.listdir(tracker_dir)[0]))
test.head()

,TIMESTAMP,UPLOAD_DATE,DOWNLOADED,DOWNLOAD_DATE,URL
0,2020-02-05 00:00:00,2020-02-06 04:40:00,False,1900-01-01 00:00:00,http://nemweb.com.au/Reports/Current/Operation...
1,2020-02-06 00:00:00,2020-02-07 04:40:00,False,1900-01-01 00:00:00,http://nemweb.com.au/Reports/Current/Operation...
2,2020-02-07 00:00:00,2020-02-08 04:40:01,False,1900-01-01 00:00:00,http://nemweb.com.au/Reports/Current/Operation...
3,2020-02-08 00:00:00,2020-02-09 04:40:00,False,1900-01-01 00:00:00,http://nemweb.com.au/Reports/Current/Operation...
4,2020-02-09 00:00:00,2020-02-10 04:40:01,False,1900-01-01 00:00:00,http://nemweb.com.au/Reports/Current/Operation...


In [4]:
nem_trk.resources.keys()

dict_keys(['/Reports/Current/Operational_Demand/ACTUAL_DAILY/'])